# Syntax Improvements to Pixel-Flipping Algorithm

Compare the syntax of masks with loops for the next iteration of pixel-flipping which should take coarser steps.

## Random number generation

### Numpy

In [1]:
from numpy.random import default_rng
rng = default_rng(seed=42)
rng.uniform(low=-1, high=1)

0.5479120971119267

### PyTorch

In [2]:
import torch

# Random Number Generator
ran_num_gen = torch.Generator()
ran_num_gen.manual_seed(42)

## Playground

### Removing multiple elements at a time using masks

Use pseudocode from Grégoire.

```python
# R is tensor of relevance scores
# s is tensor of sorted relevance scores MRoF
# 100 # elements to flip
# a is flip value (uniformly sampled e.g.)
s = sort(R)
R[R>s[-100]] = a
```

In [3]:
import torch

ran_num_gen = torch.Generator()
ran_num_gen.manual_seed(42)

# Random relevance scores
R = torch.randint(low=0, high=10, size=(10,), generator=ran_num_gen)
print(f'R:                {R}')

s_values, s_indices = R.sort(dim=-1, descending=True, stable=True)

print(f'sorted R values:  {s_values}')
print(f'sorted R indices: {s_indices}')

n_flips = 4
# n_flips-1 because indexing starts at 0
print(f'Bottom threshold to flip—i.e., all values >= {s_values[n_flips-1]} will be flipped.')
print(f'Elems. to flip: {s_values[:n_flips]}')
print(f'Mask notation of elems. to flip: {R[R>s_values[n_flips]]}')

R:                tensor([2, 7, 6, 4, 6, 5, 0, 4, 0, 3])
sorted R values:  tensor([7, 6, 6, 5, 4, 4, 3, 2, 0, 0])
sorted R indices: tensor([1, 2, 4, 5, 3, 7, 9, 0, 6, 8])
Bottom threshold to flip—i.e., all values >= 5 will be flipped.
Elems. to flip: tensor([7, 6, 6, 5])
Mask notation of elems. to flip: tensor([7, 6, 6, 5])


### Compare fine-grained with two implementation types

1. Using a mask to select the elements to flip
1. Selecting the elements sequentially (in a loop)

Tensor `R` is a tensor of relevance scores of size `N` with values between `RANDOM_LOW` and `RANDOM_HIGH`.
We're going to flip `K` elements of this tensor and replace them by a single value from a uniform distribution.

#### Setup

In [4]:
# Number of relevance scores in R
N = 10
# Number of flips
K = 5
# Parameters for random number generation
SEED = 42
RANDOM_LOW = 0
RANDOM_HIGH = 100

In [5]:
import torch

torch_rng = torch.Generator()
torch_rng.manual_seed(SEED)

from numpy.random import default_rng

numpy_rng = default_rng(seed=SEED)
numpy_rng_flip_val = numpy_rng.uniform(low=RANDOM_LOW, high=RANDOM_HIGH)

R = torch.randint(low=RANDOM_LOW, high=RANDOM_HIGH, size=(N,), generator=torch_rng)
r_mask = R.clone().detach()
r_seq = R.clone().detach()

##### Sort relevance scores

In [6]:
s_values, s_indices = R.sort(dim=-1, descending=True, stable=True)

Debug

In [7]:
print(f'numpy_rng_flip_val: {int(numpy_rng_flip_val)}')
print(f'sorted R values:  {s_values}')
print(f'sorted R indices:  {s_indices}')
print(f'elems. to flip: {s_values[:K]}')

numpy_rng_flip_val: 77
sorted R values:  tensor([76, 67, 50, 42, 35, 26, 24, 20, 14, 13])
sorted R indices:  tensor([2, 1, 8, 0, 5, 4, 7, 6, 3, 9])
elems. to flip: tensor([76, 67, 50, 42, 35])


#### Flip using mask

In [8]:
print(f'R before: {r_mask}')
r_mask[r_mask>s_values[n_flips]] = numpy_rng_flip_val
print(f'R after: {r_mask}')

R before: tensor([42, 67, 76, 14, 26, 35, 20, 24, 50, 13])
R after: tensor([77, 77, 77, 14, 26, 35, 20, 24, 77, 13])


#### Flip sequentially (in a loop)

Caveat: If the flip value is larger than the highest relevance score, the loop only changes one single value.

1. First iteration: Replace max. relevance score by flip value
1. Second iteration: Replace second max. relevance score by flip value, but max. relevance score is now the flip value. Thus, no changes will be made.

In [9]:
print(f'R before: {r_seq}')

for i in range(K):
  # print(f'Step {i+1}\n')

  # 1. Sort by relevance
  # Get indices of max value in relevance scores
  r_max = (r_seq==torch.max(r_seq)).nonzero()
  j = r_max.flatten().item()

  r_seq[j] = numpy_rng_flip_val

print(f'R after: {r_seq}')

R before: tensor([42, 67, 76, 14, 26, 35, 20, 24, 50, 13])
R after: tensor([42, 67, 77, 14, 26, 35, 20, 24, 50, 13])
